In [1]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
import matplotlib.pyplot as plt
import plotly as py
from shapely.geometry import Point, Polygon
import plotly.express as px

Paris is hosting the Paralympics in 2024. One of the events at the Paralympics is an open water
swimming event in the Seine (apparently the water is clean or so the French say). Since the City
of Amsterdam thinks it is beter than Paris, they want to host an event before the Paralympics,
snubbing the Parisians. The idea is to host a 5km. open water swimming event through the
canals of Amsterdam. You are asked by the municipality of Amsterdam to advise on the
feasibility of the event from the perspective of the safety of the partaking athletes from an
environmental perspective. The event is going to be hosted in May.

For this exercise you need the data from this website. Download the Basis Bestand Gebieden
Amsterdam (BBGA), both the data and the documentation. You already have the AirBnB data
from Amsterdam from the workshop exercise.

The Municipality of Amsterdam is in a love hate relationship with AirBnb, see for example this
and this article. Amsterdam wants to get a bit of insight in the number of tourists that will
make use of AirBnB. Can you advise on or calculate for Amsterdam:

- What Amsterdam will receive from tourist tax if the event lasts a week and you will
have 30.000 visitors?

In [2]:
tourist_tax = 3 * 30000 * 6
print(tourist_tax, "euros")

540000 euros


- Plot the amount of AirBnB locations per neighbourhood

In [3]:
airbnb_df = pd.read_csv('./listings.csv')
bbga_df = pd.read_csv('./bbga.csv', sep=';')

print(bbga_df.head())

C:\Users\julia\AppData\Local\Temp\ipykernel_33192\52587493.py:2: DtypeWarning: Columns (15,16,17,18,21,22,26,28,32,43,44,45,46,47,48,49,50,51,53,65,66,67,68,69,72,78,87,88,89,111,119,121,123,125,127,130,132,136,138,140,143,145,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,197,198,199,206,207,208,209,210,211,212,214,216,222,227,240,241,242,243,244,245,248,249,250,254,255,256,257,258,259,262,263,264,265,266,267,273,276,277,278,279,280,281,282,283,284,285,298,299,300,301,302,303,304,333,335,336,337,338,342,343,360,361,362,364,367,369,405,442,443,447,448,450,454,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,494,495,496,497,498,499,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,668,674,676,684,702,703,704,705,708,710,711,712,714,716,718,720,722,724,736,737,738,739,741,742,757,758,759,760,761,769,770,771,772,776,7

   niveau niveaunaam    sd          sdnaam gebiedcode22 gebiedcodenaam  \
0     1.0   Gemeente  STAD  0363 Amsterdam         STAD      Amsterdam   
1     2.0  Stadsdeel     A       A Centrum            A      A Centrum   
2     2.0  Stadsdeel     B     B Westpoort            B    B Westpoort   
3     2.0  Stadsdeel     E          E West            E         E West   
4     2.0  Stadsdeel     F    F Nieuw-West            F   F Nieuw-West   

   gebiednaam  gsd  jaar  BEVTOTAAL  ...  SK017_KWETS34  SK017_KWETS34_P  \
0   Amsterdam  NaN  2023   918193.0  ...            NaN              NaN   
1     Centrum  NaN  2023    90020.0  ...            NaN              NaN   
2   Westpoort  NaN  2023     1009.0  ...            NaN              NaN   
3        West  NaN  2023   148569.0  ...            NaN              NaN   
4  Nieuw-West  NaN  2023   163448.0  ...            NaN              NaN   

   SK1826_KWETS34  SK1826_KWETS34_P  SK2765_KWETS34 SK2765_KWETS34_P  \
0             NaN         

In [4]:
df_neighbourhood = airbnb_df['neighbourhood'].value_counts().reset_index().rename(columns={'index': 'neighbourhood', 0 : 'count'})
print(df_neighbourhood)

                             neighbourhood  count
0                   De Baarsjes - Oud-West   1382
1                             Centrum-West   1006
2                  De Pijp - Rivierenbuurt    935
3                             Centrum-Oost    793
4                                     Zuid    599
5                               Westerpark    584
6                                 Oud-Oost    506
7                            Bos en Lommer    424
8                                Oud-Noord    402
9   Oostelijk Havengebied - Indische Buurt    320
10                         Watergraafsmeer    262
11                              Noord-West    218
12                IJburg - Zeeburgereiland    183
13                             Slotervaart    164
14                              Noord-Oost    137
15                 Geuzenveld - Slotermeer    116
16                  Buitenveldert - Zuidas    100
17                  De Aker - Nieuw Sloten     74
18                   Gaasperdam - Driemond     51


In [5]:
figure = px.bar(df_neighbourhood, x='neighbourhood', y='count')
figure.show()

- Which street in Amsterdam has the most AirBnB apartments

- Try to cross reference the data from the AirBnB dataset with the BBGA. Can you figure
out if all apartments of AirBnB are designated as housing? Which number of
apartments are not rented out all the time but are also used as normal housing?

In [20]:
row_name = {"Oud West, De Baarsjes" : "De Baarsjes - Oud-West",
            "De Pijp, Rivierenbuurt" : "De Pijp - Rivierenbuurt",
            "Indische Buurt, Oostelijk Havengebied" : "Oostelijk Havengebied - Indische Buurt",
            "IJburg, Zeeburgereiland" : "Ijburg - Zeeburgereiland",
            "Geuzenveld, Slotermeer" : "Geuzenveld - Slotermeer",
            "Buitenveldert, Zuidas" : "Buitenveldert - Zuidas",
            "Oud-Zuid" : "Zuid" ,
            "De Aker, Sloten, Nieuw-Sloten" : "De Aker - Nieuw Sloten"}

bbga_airbnb_df = bbga_df[['gebiednaam', 'sdnaam', 'BHVESTAIRBNB']]

bbga_airbnb_df_GWW = bbga_airbnb_df.iloc[3937:3962]

bbga_airbnb_df_GWW = bbga_airbnb_df_GWW.rename(index=row_name)

print(bbga_airbnb_df_GWW)

                                 gebiednaam      sdnaam  BHVESTAIRBNB
3937                           Centrum-West     Centrum        3525.0
3938                           Centrum-Oost     Centrum        2766.0
3939                             Westerpark        West        2125.0
3940                          Bos en Lommer        West        1551.0
3941                  Oud West, De Baarsjes        West        4868.0
3942                  Sloterdijk Nieuw-West  Nieuw-West          18.0
3943                 Geuzenveld, Slotermeer  Nieuw-West         375.0
3944                                 Osdorp  Nieuw-West         270.0
3945          De Aker, Sloten, Nieuw-Sloten  Nieuw-West         204.0
3946                            Slotervaart  Nieuw-West         616.0
3947                               Oud-Zuid        Zuid        2045.0
3948                  Buitenveldert, Zuidas        Zuid         408.0
3949                 De Pijp, Rivierenbuurt        Zuid        3593.0
3950                

In [21]:
graph = px.bar(bbga_airbnb_df_GWW, x='gebiednaam', y='BHVESTAIRBNB')
graph.add_bar(x=df_neighbourhood['neighbourhood'], y=df_neighbourhood['count'])

graph.show()

- How many hotel rooms should be built if Amsterdam wants to accommodate the same
number of tourists?

- How many different licenses are issued?

In [ ]:
airbnb_df

licence = list(set(licence)) 
Print (“The number of licences after removing duplicates : “ + str(licence))